In [2]:
import math
import numpy as np

In [ ]:
#Setting the constants in a way that contstraints of Theorem 4.1 are satisfied
#Here I defined f_i's to f_i(x) := x^2 + 3sin(x)^2 (which is a non-convex function that satisfies PL-Condition)

#Value of the f_i at point x(w.r.t. defined f_i)
def get_f(x, n):
    f = np.zeros(n)
    for i in range(n):
        f[i] = x[i] ** 2 + 3 * math.sin(x[i]) ** 2
    return f
#Derivative of function at the point x(w.r.t. defined f_i)
def drv(x, n):
    grd = np.zeros(n)
    for i in range(n):
        grd[i] = 2 * x[i] + 3 * math.sin(2 * x[i])
    return grd

In [3]:
mu = 1 / 32 #Lower bound constant related to PL-Condition(w.r.t. defined f_i)
L = 3.67067936 #Upper bound constant related to Lipschitz Gradient Condition(w.r.t defined f_i)(foud this constant experimentally)
etta = 1 / (2 * L * math.sqrt(L)) #Learning rate of the GD-Algorithm
D = 1557 #Problem dimension(same as the one used in the paper), options: 1557, 6217, 24849
K = 5 #Number of agents(same as the one used in the paper)
C = max(math.sqrt(16 * L / (mu ** 2) + L * math.sqrt(L)) / mu, 100) #Constant related to Algorithm 1(same as the one used in Theorem 4.1)
b = 64 #Number of bits to represent each entry

#Amount of effect each agent's estimation takes from other agents estimation of minimizer at time t
def gamma(t):
    return 1 / t

theta_Main = np.zeros(D) #Main estimation of the minimizer
theta = np.zeros((D, K)) #Each Agent's estimation of the minimizer
g = np.zeros((D, K)) #Compressed gradient of each agent's value at the estimated point
q = np.zeros((D, K)) #Quantized value for each agent's estimation of minimizer

In [ ]:
#Quantization, Compression and Decompression

#Compression function(same as the one used in Theorem 4.1)
def F_Compressor(x):
    return x

#Decompression function(same as the one used in Theorem 4.1)
def F_tilde_Decompressor(x):
    return x

#Quantization distance unit
def delta(r, b):
    return r / (2 ** b - 1)

def quant(c, p, r, j, n):
    for i in range(n):
        if p[i] <= c[i] - r + delta(r, b):
            p[i] = p[i] - r
        elif p[i] >= c[i] + r - delta(r, b):
            p[i] = p[i] + r
        else:
            p[i] = p[i] - r + 2 * delta(r, b) * math.floor((c[i] - p[i] + r + delta(r, b)) / 2 * delta(r, b))
    return p

In [ ]:
#Main part of the Algorithm

for t in range(1000):
    #Performing each agent's work
    for i in range(K):
        sum = np.zeros(K) #Sum of the q_j's
        for j in range(K):
            sum = sum + q[i]
        theta[i] = theta_Main - etta * F_tilde_Decompressor(sum) #Update
        theta_Main = theta[i] #Update
        q[i] = quant(F_Compressor(drv(theta_Main, D)), q[i], gamma(t), b, D) #Quantize

print(theta_Main, get_f(theta_Main))